# Неперсонализованные рекомендации (Non-Personalised Recommendations)

Когда пользователь только приходит на сервис мы ничего не знаем о его вкусах, так как откликов от него еще не было. В таком случае применяют неперсонализованные рекомендации.

### Популярные рекомендации

Первая мысль которая может прийти какой товар порекомендовать - это порекомендовать популярный товар. Но тогда встает вопрос - что значит популярный?
Это может значить например:
<ul>
    <li>наиболее просматриваемый</li>
    <li>наиболее кликабельный</li>
    <li>наиболее покупаемый</li>
    <li>товары которые берут чаще всего с текущими в вашей корзине покупок</li>
</ul>
Выбор определения зависит от конкретного сервиса и каких целей вы приследуете 

In [43]:
import pandas as pd

In [45]:
df_ratings = pd.read_csv('data/ml-latest-small/ratings.csv', sep=',')

In [46]:
df_movies = pd.read_csv('data/ml-latest-small/movies.csv')

In [47]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [51]:
df_ratings['timestamp'].max()

1537799250

In [52]:
df_ratings['timestamp'] = pd.to_datetime(df_ratings['timestamp'], unit='s').sort_values()

In [53]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        #выдаем фильмы которым чаще всего ставят оценку
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column]\
                                 .value_counts()\
                                 .head(self.max_K)\
                                 .index.values
    
    def recommend(self,  N=10):
        return self.recommendations[:N]

In [58]:
recommender = PopularRecommender(days=40, item_column='movieId', dt_column='timestamp')

In [59]:
recommender.fit(df_ratings)

In [60]:
recommender.recommend()

array([122916,  58559, 122912,   8961,   5952,  33794,   6539,   7153,
        79132,   4993])

In [61]:
pd.DataFrame(recommender.recommend(), columns=['movieId']).merge(df_movies)

,movieId,title,genres
0,122916,Thor: Ragnarok (2017),Action|Adventure|Sci-Fi
1,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
2,122912,Avengers: Infinity War - Part I (2018),Action|Adventure|Sci-Fi
3,8961,"Incredibles, The (2004)",Action|Adventure|Animation|Children|Comedy
4,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy
5,33794,Batman Begins (2005),Action|Crime|IMAX
6,6539,Pirates of the Caribbean: The Curse of the Bla...,Action|Adventure|Comedy|Fantasy
7,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy
8,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
9,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


## Демографические группы, сегменты популярных рекоммендаций 

Однако легко может случиться так, что популярные продукты не соответствуют конкретно вашим вкусам. Но вы можете принадлежать некоторой демографической группе, что может хорошо определить ваши вкусы.

Если в начале у нас имеется информация о самом пользователе, такие как:
<ul>
    <li>Пол</li>
    <li>Возраст</li>
    <li>Семейный статус</li>
    <li>Социальный статус</li>
    <li>Вид и область занятости</li>
    <li>Местоположение</li>
    <li>итд</li>
</ul>
Можно провести аналитики на отдельных группах и подобрать более специфичную

### Всегда ли полезны популярные рекоммендации?

Нет, не всегда. Если подумать, то ваши вкусы вообще говоря могут и не совпадать с большинством. Например вам может нравиться старый рок, а от новых рэп альбомов вы не в восторге. Другим недостатком, когда популярные товары будут плохой рекоммендацией - их очевидность. Если вы порекомендуете популярные товары в супермаркете, то скорее всего это будут бананы, молоко, хлеб, пакет, гречка итд. Иными словами что-то что вы бы и так купили, а рекомендация полностью бесполезна и ничего нового вам не предложила

### Assosiation rules

Другим простым, но эффективным способом порекомендовать товары - это посмотреть какие товары чаще встречаются в одной корзине (транзакции, последовательности действий). В данном случае мы пытаемся определить есть ли взаимосвязь в имеющихся данных продуктов. Рекомендации такого вида можно описать как "С этими товарами также берут", "Те кто купил X, также купили Y"

<img src="images/cust_who_bought.png" alt="Сustomers Who bought" style="width: 600px;"/>

Для начала введем несколько определений которые будут описывать эти правила

Пусть T - множество транзакций, X, Y - подмножестов товаров

In [62]:
df = pd.DataFrame({
    'transaction_id': list(range(5)),
    'milk': [1, 0, 0, 1, 0],
    'bread': [1, 0, 0, 1, 1],
    'butter': [0, 1, 0, 1, 0],
    'beer': [0, 0, 1, 0, 0],
    'diapers': [0, 0, 1, 0, 0]})

In [63]:
df.head()

,transaction_id,milk,bread,butter,beer,diapers
0,0,1,1,0,0,0
1,1,0,0,1,0,0
2,2,0,0,0,1,1
3,3,1,1,1,0,0
4,4,0,1,0,0,0


### Support
$$ supp(X) = \frac{|\{X\subset{T}\}|}{|T|} abc$$
Support - частотность транзакций с подмножестовм товаров ко всем транзакциям. Стоит отметить, что нотация $supp(x_1\cup{x_2})$ будет означать частотность транзакций, когда товары $x_1$ и $x_2$ встречаются транзакции вместе

In [14]:
def support(df, ascendents):
    pass

assert support(df, ['milk']) == 0.4
assert support(df, ['butter', 'bread']) == 0.2

### Confidence
$$ conf(X->Y)=\frac{supp(X\cup{Y})}{supp(X)}$$
Confidence (достоверность) - показатель того, как часто правило X -> Y срабатывает для всего датасета. Или более математическим способом P(Y | X)

In [15]:
ascendent = X
conscendent = Y
def confidence(df, ascendents, conscendents):
    # code here
    pass

assert confidence(df, ['bread', 'butter'], ['milk']) == 1 # правило корректно в 100% случаев
assert confidence(df, ['butter'], ['bread']) == 0.5 # правило срабатывает в 50% случаев

### Lift
$$Lift(X->Y)=\frac{supp(X\cup{Y})}{supp(X)*supp(Y)}$$
Lift (поддержка) - показатель того, что X и Y являются независимыми наборами товаров.
* Если lift = 1 утверждается, что появление товаров независимо и собственно правила здесь нет
* Если lift < 1, это позволяет нам понять, что элементы заменяют друг друга. Это означает, что наличие одного элемента отрицательно влияет на наличие другого элемента и наоборот.
* Если lift > 1, то рассматриваемое правило потенциально полезно и чем lift выше, тем "сильнее правило"

In [16]:
def lift(df, ascendents, conscencdents):
    # code here
    pass

assert lift(df, ['milk', 'bread'], ['butter']) == 1.25

На практике нам нужно найти сами правила которые удовлетворяют заданным порогам по support и lift. Для этого есть несколько алгоритмов: *BruteForce*, *Apriori algorythm*, *ECLAT Algorithm* и *FP-Growth Algorithm*. Для примера мы используем реализацию алгоритма apriory из библиотеки mlxtend.

<img src="images/apriory.png" alt="Сustomers Who bought" style="width: 600px;"/>

Если коротко про алгоритм, то он строит дерево перебора правил и рассчитывает support для них. Если посчитанный support для правила не проходит по порогу, то любое супермножество этого правило далее не рассматриваются

Подробнее о метриках выше и алгоритмах вы можете почитать здесь
https://habr.com/ru/company/ods/blog/353502/#4

In [17]:
# !pip install mlxtend  

In [64]:
from mlxtend.frequent_patterns import (apriori, association_rules)

In [65]:
df = pd.read_csv('data/test.csv')

In [66]:
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
2,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam


In [67]:
df['Item'].value_counts()

Coffee            5471
Bread             3325
Tea               1435
Cake              1025
Pastry             856
                  ... 
Raw bars             1
Polenta              1
Olum & polenta       1
The BART             1
Gift voucher         1
Name: Item, Length: 95, dtype: int64

In [68]:
# убираем повторения и пропущенные значения
df = df.sort_values(['Date', 'Time', 'Transaction'])
df.drop_duplicates(inplace=True)
df = df[df['Item'] != "NONE"]

In [69]:
df.head()

,Date,Time,Transaction,Item
0,2016-10-30,09:58:11,1,Bread
1,2016-10-30,10:05:34,2,Scandinavian
3,2016-10-30,10:07:57,3,Hot chocolate
4,2016-10-30,10:07:57,3,Jam
5,2016-10-30,10:07:57,3,Cookies


In [36]:
df['Transaction'].nunique()

9465

In [70]:
df['count'] = 1
df_transactions = df.groupby(['Transaction', 'Item'])['count'].sum().unstack().fillna(0)

In [71]:
df_transactions

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Bare Popcorn,Basket,...,The BART,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
freq_items = apriori(df_transactions, min_support=0.01, use_colnames=True)

In [73]:
freq_items.sort_values('support', ascending=False)

,support,itemsets
6,0.478394,(Coffee)
2,0.327205,(Bread)
26,0.142631,(Tea)
4,0.103856,(Cake)
34,0.090016,"(Bread, Coffee)"
...,...,...
11,0.010565,(Hearty & Seasonal)
20,0.010460,(Salad)
30,0.010354,"(Alfajores, Bread)"
58,0.010037,"(Cake, Bread, Coffee)"


In [74]:
rules = association_rules(freq_items, metric='lift', min_threshold=1)

In [76]:
rules.sort_values('lift', ascending=False)[:20]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
40,(Cake),"(Tea, Coffee)",0.103856,0.049868,0.010037,0.096643,1.937977,0.004858,1.051779
39,"(Tea, Coffee)",(Cake),0.049868,0.103856,0.010037,0.201271,1.937977,0.004858,1.121962
8,(Cake),(Hot chocolate),0.103856,0.058320,0.011410,0.109868,1.883874,0.005354,1.057910
9,(Hot chocolate),(Cake),0.058320,0.103856,0.011410,0.195652,1.883874,0.005354,1.114125
11,(Tea),(Cake),0.142631,0.103856,0.023772,0.166667,1.604781,0.008959,1.075372
10,(Cake),(Tea),0.103856,0.142631,0.023772,0.228891,1.604781,0.008959,1.111865
30,(Toast),(Coffee),0.033597,0.478394,0.023666,0.704403,1.472431,0.007593,1.764582
31,(Coffee),(Toast),0.478394,0.033597,0.023666,0.049470,1.472431,0.007593,1.016699
37,(Pastry),"(Bread, Coffee)",0.086107,0.090016,0.011199,0.130061,1.444872,0.003448,1.046033
36,"(Bread, Coffee)",(Pastry),0.090016,0.086107,0.011199,0.124413,1.444872,0.003448,1.043749
